In [1]:
import urllib
import pandas as pd
import time
from tqdm.notebook import tqdm
import re
import wget

# extract doi list

In [2]:
all_abstracts = pd.read_csv('data/included_abstracts.csv', index_col = 0)
len(all_abstracts)

42587

In [3]:
all_abstracts['article_date'] = pd.to_datetime(all_abstracts['article_date'])
decade_df = all_abstracts[(all_abstracts['article_date'] > '2012-01-01') & (all_abstracts['article_date'] <'2022-01-01')]
len(decade_df)

28703

In [4]:
decade_df.sort_values(by='article_date',  ascending=False, inplace=True)
decade_df.head(50)

C:\Users\Joe Z\anaconda3\envs\scraper\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,pmid,doi,title,abstract,article_date,pubmed_date,article_type,lang,journal,journal_short,...,spec_renal,affil_countries,affil_countries_unique,affil_first_country,affil_last_country,affil_fill_country,countries_lc,lmic_author_flag,lmic_author_lower_flag,lmic_china_flag
23767,35011970,10.3390/jcm11010229,An Application of Machine Learning That Uses t...,This retrospective single-center study include...,2021-12-31 00:00:00+00:00,2022-01-12 00:00:00+00:00,Journal Article,eng,Journal of clinical medicine,J Clin Med,...,0.0,"['South Korea', 'South Korea', 'South Korea', ...",['South Korea'],South Korea,South Korea,South Korea,['south korea'],0.0,0.0,0.0
34412,34972109,10.1371/journal.pone.0261698,DAVS-NET: Dense Aggregation Vessel Segmentatio...,"In this era, deep learning-based medical image...",2021-12-31 00:00:00+00:00,2022-01-01 00:00:00+00:00,Journal Article,eng,PloS one,PLoS One,...,0.0,"['Pakistan', 'Pakistan', 'Pakistan', 'Saudi Ar...","['Australia', 'Pakistan', 'Russia', 'Saudi Ara...",Pakistan,Pakistan,Pakistan,"['australia', 'pakistan', 'russia', 'saudi ara...",1.0,0.0,0.0
29263,34971557,10.1371/journal.pone.0260600,Optical coherence tomography for identificatio...,To explore the feasibility of using random for...,2021-12-31 00:00:00+00:00,2022-01-01 00:00:00+00:00,Journal Article,eng,PloS one,PLoS One,...,0.0,"['China', 'China', 'China', 'China', 'China', ...",['China'],China,China,China,['china'],1.0,0.0,1.0
26000,35005197,10.1002/dad2.12264,Deep learning improves utility of tau PET in t...,Positron emission tomography (PET) imaging tar...,2021-12-31 00:00:00+00:00,2022-01-11 00:00:00+00:00,Journal Article,eng,"Alzheimer's & dementia (Amsterdam, Netherlands)",Alzheimers Dement (Amst),...,0.0,[nan],[nan],NaN,NaN,United States,[nan],0.0,0.0,0.0
39158,34969093,10.1093/cercor/bhab474,Predicting Superagers by Machine Learning Clas...,Superagers are defined as older adults who hav...,2021-12-31 00:00:00+00:00,2021-12-31 00:00:00+00:00,Journal Article,eng,"Cerebral cortex (New York, N.Y. : 1991)",Cereb Cortex,...,0.0,"['South Korea', 'South Korea', 'South Korea', ...",['South Korea'],South Korea,South Korea,South Korea,['south korea'],0.0,0.0,0.0
42411,35005676,10.1016/j.cvdhj.2021.12.003,Identifying Risk of Adverse Outcomes in COVID-...,Adverse events in COVID-19 are difficult to pr...,2021-12-31 00:00:00+00:00,2022-01-11 00:00:00+00:00,Journal Article,eng,Cardiovascular digital health journal,Cardiovasc Digit Health J,...,0.0,"['United States', 'United States', 'United Sta...","['Norway', 'United States', 'Sweden', 'Denmark']",United States,United States,United States,"['norway', 'united states', 'sweden', 'denmark']",0.0,0.0,0.0
18320,35008361,10.3390/cancers14010188,A Clinical Prognostic Model Based on Machine L...,Multicenter clinical trials are producing grow...,2021-12-31 00:00:00+00:00,2022-01-12 00:00:00+00:00,Journal Article,eng,Cancers,Cancers (Basel),...,0.0,"['Italy', 'Italy', 'Germany', 'Italy', 'Italy'...","['Germany', 'Netherlands', 'France', 'Italy']",Italy,Italy,Italy,"['germany', 'netherlands', 'france', 'italy']",0.0,0.0,0.0
542,35003258,10.1155/2021/5499385,Deep Learning to Predict EGFR Mutation and PD-...,The detection of epidermal growth factor recep...,2021-12-31 00:00:00+00:00,2022-01-11 00:00:00+00:00,Journal Article,eng,Journal of oncology,J Oncol,...,0.0,"['China', 'China', 'China', 'China', 'China', ...",['China'],China,China,China,['china'],1.0,0.0,1.0
20236,34998166,10.1016/j.cmpb.2021.106613,TMSF-Net: Multi-series fusion network with tre...,Colorectal tumors are common clinical diseases...,2021-12-31 00:00:00+00:00,2022-01-09 00:00:00+00:00,Journal Article,eng,Computer methods and programs in biomedicine,Comput Methods Programs Biomed,...,0.0,"['China', 'China', 'China', 'China', 'China', ...",['China'],China,China,China,['china'],1.0,0.0,1.0
16453,35011959,10.3390/jcm11010219,Machine Learning to Calculate Heparin Dose in ...,To realize a machine learning (ML) model to es...,2021-12-31 00:00:0

In [5]:
id_dict = dict(zip(decade_df['pmid'], decade_df['doi']))
pmid_list = decade_df['pmid'].tolist()
doi_list = decade_df['doi'].tolist()

## optionally manually chunk for retrieval as website api arbitrarily gets cross 

In [6]:
chunk_list = doi_list

#chunk_list = doi_list[0:5000]
#chunk_list_2 = doi_list[5000:10000]
#chunk_list_3 = doi_list[10000:15000]
#chunk_list_4 = doi_list[15000:20000]
#chunk_list_5 = doi_list[20000:28703]

# obtain list of urls for direct pdf access

In [8]:
#api key for openaccessbutton.org
api_key = "e5fadce50b8396f6c398a43ed50c68"

In [9]:
#create list of openaccessbutton.org urls
url_list = []

for doi in chunk_list:
    try:
        doi_encoded = urllib.parse.quote(doi, safe='')
    except:
        doi_encoded = ("nan")
    
    link = "https://api.openaccessbutton.org/find?id={}&apikey={}".format(doi_encoded, api_key)
    
    url_list.append(link)

url_list

['https://api.openaccessbutton.org/find?id=10.3390%2Fjcm11010229&apikey=e5fadce50b8396f6c398a43ed50c68',
 'https://api.openaccessbutton.org/find?id=10.1371%2Fjournal.pone.0261698&apikey=e5fadce50b8396f6c398a43ed50c68',
 'https://api.openaccessbutton.org/find?id=10.1371%2Fjournal.pone.0260600&apikey=e5fadce50b8396f6c398a43ed50c68',
 'https://api.openaccessbutton.org/find?id=10.1002%2Fdad2.12264&apikey=e5fadce50b8396f6c398a43ed50c68',
 'https://api.openaccessbutton.org/find?id=10.1093%2Fcercor%2Fbhab474&apikey=e5fadce50b8396f6c398a43ed50c68',
 'https://api.openaccessbutton.org/find?id=10.1016%2Fj.cvdhj.2021.12.003&apikey=e5fadce50b8396f6c398a43ed50c68',
 'https://api.openaccessbutton.org/find?id=10.3390%2Fcancers14010188&apikey=e5fadce50b8396f6c398a43ed50c68',
 'https://api.openaccessbutton.org/find?id=10.1155%2F2021%2F5499385&apikey=e5fadce50b8396f6c398a43ed50c68',
 'https://api.openaccessbutton.org/find?id=10.1016%2Fj.cmpb.2021.106613&apikey=e5fadce50b8396f6c398a43ed50c68',
 'https://a

In [10]:
url_start = 'url": "'
url_end = '",'
pdf_list = []

timer_1 = time.time()

for url in tqdm(url_list):
    #avoid timeout
    time.sleep(5)
    try:
        #api request from openaccessbutton.org
        content = urllib.request.urlopen(url)
        #parse out pdf url if it exists
        url_text = str(content.read())
        
        try:
            pdf_list.append(url_text.split(url_start)[1].split(url_end)[0])
        except:
            pdf_list.append('nan')
    
    except:
        pdf_list.append('nan')

timer_2 = time.time()
print("time taken:")
print(timer_2 - timer_1)

  0%|          | 0/5000 [00:00<?, ?it/s]

time taken:
27566.508113622665


In [11]:
pdf_list

['https://www.mdpi.com/2077-0383/11/1/229/pdf',
 'https://journals.plos.org/plosone/article/file?id=10.1371/journal.pone.0261698&type=printable',
 'https://journals.plos.org/plosone/article/file?id=10.1371/journal.pone.0260600&type=printable',
 'https://onlinelibrary.wiley.com/doi/pdfdirect/10.1002/dad2.12264',
 'nan',
 'http://www.cvdigitalhealthjournal.com/article/S2666693621001468/pdf',
 'https://www.mdpi.com/2072-6694/14/1/188/pdf',
 'https://downloads.hindawi.com/journals/jo/2021/5499385.pdf',
 'nan',
 'https://www.mdpi.com/2077-0383/11/1/219/pdf',
 'https://bmjopen.bmj.com/content/bmjopen/11/12/e053808.full.pdf',
 'nan',
 'nan',
 'nan',
 'https://www.mdpi.com/1424-8220/22/1/299/pdf',
 'http://arxiv.org/pdf/2005.07229',
 'https://downloads.hindawi.com/journals/cin/2021/5759184.pdf',
 'https://www.mdpi.com/1424-8220/22/1/283/pdf',
 'nan',
 'https://doi.org/10.1088/1361-6560/ac4316',
 'nan',
 'https://www.mdpi.com/1424-8220/22/1/292/pdf',
 'https://doi.org/10.1016/j.compbiomed.2021.

# pair pmid to pdf 

In [1]:
#len(pdf_list)

In [2]:
name_list = pmid_list
name_list = [str(item) for item in name_list]
name_list = ["open_access/" + item for item in name_list]
print(len(name_list))

In [14]:
pdf_urls = pd.DataFrame(
    {'name': name_list,
     'pdf_url': pdf_list,
    })

#pdf_urls.to_csv('output/openaccesslinks/pdf_urls_0_5k.csv')

# join files and split to open access/non-open access

In [ ]:
#df01 = pd.read_csv('output/pdf_urls_0_5k.csv', index_col = 0)
#df02 = pd.read_csv('output/pdf_urls_5k_6k.csv', index_col = 0)
#df03 = pd.read_csv('output/pdf_urls_6k_10k.csv', index_col = 0)
#df04 = pd.read_csv('output/pdf_urls_10k_15k.csv', index_col = 0)
#df05 = pd.read_csv('output/pdf_urls_15k_28k.csv', index_col = 0)

#df_links = pd.concat([df01, df02, df03, df04, df05], ignore_index=True)

df_links = pdf_urls.copy()
df_links.head()

In [ ]:
df_links['pmid'] = df_links['name'].str.replace("open_access/", "")
df_links.head()

In [ ]:
# add doi column

df_links['doi'] = df_links['pmid'].map(id_dict)
df_links.head()

In [ ]:
open_links = df_links[~df_links['pdf_url'].isnull()]
no_links = df_links[df_links['pdf_url'].isnull()] 
print('links present')
print(len(open_links))
print('no links')
print(len(no_links))

In [ ]:
open_links.to_csv('output/open_access_links.csv')
no_links.to_csv('output/no_links_for_open_access.csv')